In [48]:
library(tidyverse)

df <- read_csv("https://raw.githubusercontent.com/petebrown/scrape-results-playwright/main/data/records.csv", show_col_types = FALSE)

results <- read_csv("https://raw.githubusercontent.com/petebrown/update-results/main/data/results_df.csv", show_col_types = FALSE)

venues <- read_csv("https://raw.githubusercontent.com/petebrown/update-results/main/data/results_df.csv", show_col_types = FALSE) %>%
    rename(game_url = source_url) %>%
    select(game_url, venue)

In [49]:
df2 <- df %>%
    left_join(venues, by = "game_url") %>%
    mutate(
        game_date = lubridate::dmy(game_date),
        ko_time = str_match(match_notes, "(?:[Kk]ick[\\s|-][Oo]ff:\\s)(\\d+.\\d+[a|p]m)")[,2],
    ) %>%
    mutate(
        competition_2 = str_replace(competition_2, "Associate Members' Cup", "Associate Members Cup"),
        cup_stage = str_match(competition_2, str_glue("{competition}\\s(.*)"))[,2],
        cup_round_no = str_match(competition_2, "(\\d+).*\\sround")[,2],
        cup_round = case_when(
            str_detect(cup_stage, regex("quarter-final", ignore_case = TRUE)) ~ "QF",
            str_detect(cup_stage, regex("semi-final", ignore_case = TRUE)) ~ "SF",
            str_detect(cup_stage, regex("^final", ignore_case = TRUE)) ~ "F",
            str_detect(cup_stage, regex("preliminary", ignore_case = TRUE)) ~ "P",
            str_detect(cup_stage, regex("group", ignore_case = TRUE)) ~ "G",
            .default = cup_round_no
        ),
        cup_replay = case_when(
            grepl("\\sreplay", competition_2, ignore.case = TRUE) ~ 1,
            TRUE ~ 0
        ),
        cup_section = case_when(
            str_detect(cup_stage, regex("\\s([a-zA-Z]+)\\s[Ss]ection")) ~ str_match(str_to_title(cup_stage), "\\s([a-zA-Z]+)\\s[Ss]ection")[,2],
            str_detect(cup_stage, regex("Group\\s[a-zA-Z]\\s–\\s[a-zA-Z]+")) ~ str_match(str_to_title(cup_stage), "(Group\\s[a-zA-Z]\\s–\\s[a-zA-Z]+)")[,2],
            .default = NA
        ),
        cup_leg = str_match(competition_2, "(\\d+)[a-zA-Z]+\\sleg")[,2]
    ) %>%
    mutate(
        extra_time = case_when(
            grepl("after extra time", match_notes, ignore.case = TRUE) ~ 1,
            TRUE ~ 0)
    ) %>%
    mutate(
        penalties = case_when(
            grepl("penalt", score_2, ignore.case = TRUE) ~ str_match(score_2, "(?:[Pp]enalties\\s)(\\d+-\\d+ to\\s.+)"),
            grepl("penalt", match_notes, ignore.case = FALSE) ~ str_match(match_notes, "(?:penalty\\sshoot[-|\\s]out\\s)\\((\\d+-\\d+)\\)"),
            grepl("PENALT", match_notes, ignore.case = FALSE) ~ str_match(match_notes, "(?:[^|\\|].+\\sWON\\s)(\\d+-\\d+)(?:\\sON\\sPENALTIES)"),
            .default = NA)[,2],
        pen_winners = case_when(
            grepl("\\d+-\\d+ to", penalties, ignore.case = TRUE) ~ str_match(penalties, "\\d+-\\d+\\sto\\s(.*)")[,2],
            grepl("were awarded a bonus point", match_notes, ignore.case = TRUE) ~ str_match(match_notes, ",\\s(.*)\\swere\\sawarded\\sa\\sbonus\\spoint")[,2],
            grepl("won \\d+-\\d+ on penalties", match_notes, ignore.case = TRUE) ~ str_match(str_to_title(match_notes), "(([a-zA-Z]+\\s)?[a-zA-Z]+)\\sWon\\s\\d+-\\d+\\sOn\\sPenalties")[,2],
            TRUE ~ NA
        ),
        pen_score = case_when(
            is.na(pen_winners) ~ NA,
            !grepl("Tranmere", pen_winners, ignore.case = TRUE) ~ paste0(str_match(penalties, "-(\\d+)")[,2], "-", str_match(penalties, "(\\d+)-")[,2]),
            TRUE ~ str_match(penalties, "(\\d+-\\d+)")[,2]),
        pen_outcome = case_when(
            is.na(pen_winners) ~ NA,
            grepl("Tranmere", pen_winners, ignore.case = TRUE) ~ "W",
            !grepl("Tranmere", pen_winners, ignore.case = TRUE) ~ "L"
        ),
        pen_gf = case_when(
            !is.na(pen_score) & venue == "H" ~ str_match(pen_score, "(\\d+)-")[,2],
            !is.na(pen_score) & venue == "A" ~ str_match(pen_score, "-(\\d+)")[,2],
            .default = NA
        ),
        pen_ga = case_when(
            !is.na(pen_score) & venue == "H" ~ str_match(pen_score, "-(\\d+)")[,2],
            !is.na(pen_score) & venue == "A" ~ str_match(pen_score, "(\\d+)-")[,2],
            .default = NA
        ),
        pen_gf = as.numeric(pen_gf),
        pen_ga = as.numeric(pen_ga),
    ) %>%
    mutate(
        agg_score = case_when(
            grepl("on aggregate", match_notes, ignore.case = TRUE) ~ str_match(match_notes, "(\\d+-\\d+)\\son\\saggregate")[,2],
            grepl("agg:", score, ignore.case = TRUE) ~ str_match(score, "[Aa]gg:\\s?(\\d+-\\d+)")[,2],
            .default = NA
          ),
        agg_winners = str_match(match_notes, "(([a-zA-Z]+\\s)?[a-zA-Z]+)\\swon\\s\\d+-\\d+\\son\\saggregate")[,2],
        agg_outcome = case_when(
            cup_leg == "2" & is.na(agg_winners) & !is.na(agg_score) ~ "D",
            cup_leg == "2" & !is.na(agg_winners) & str_detect(agg_winners, regex("Tranmere", ignore_case = TRUE)) ~ "W",
            cup_leg == "2" & !is.na(agg_winners) & str_detect(agg_winners, regex("Tranmere", ignore_case = TRUE), negate = TRUE) ~ "L",
            .default = NA),
        agg_score = case_when(
            agg_outcome == "D" ~ agg_score,
            agg_outcome == "W" ~ agg_score,
            agg_outcome == "L" ~ paste0(str_sub(agg_score, 3, 3), "-", str_sub(agg_score, 1, 1)),
            .default = NA
        )
    ) %>%
    mutate(
        away_goals_winner = str_match(match_notes, "(([a-zA-Z]+\\s)?[a-zA-Z]+)\\swon\\son\\sthe\\saway\\sgoals\\srule")[,2],
        away_goal_outcome = case_when(
            is.na(away_goals_winner) ~ NA,
            grepl("Tranmere", away_goals_winner, ignore.case = TRUE) ~ "W",
            !grepl("Tranmere", away_goals_winner, ignore.case = TRUE) ~ "L"
        )
    ) %>%
    mutate(
        cup_leg = case_when(
            !is.na(agg_outcome) & is.na(cup_leg) ~ "2",
            .default = cup_leg
        )
    ) %>%
    mutate(
        agg_gf = case_when(
            cup_leg == "2" & is.na(agg_winners) & !is.na(agg_score) & venue == "H" ~ str_match(agg_score, "(\\d+)-")[,2],
            cup_leg == "2" & is.na(agg_winners) & !is.na(agg_score) & venue == "A" ~ str_match(agg_score, "-(\\d+)")[,2],
            .default = NA
        ),
        agg_ga = case_when(
            cup_leg == "2" & is.na(agg_winners) & !is.na(agg_score) & venue == "H" ~ str_match(agg_score, "-(\\d+)")[,2],
            cup_leg == "2" & is.na(agg_winners) & !is.na(agg_score) & venue == "A" ~ str_match(agg_score, "(\\d+)-")[,2],
            .default = NA
        ),
        agg_gf = as.numeric(agg_gf),
        agg_ga = as.numeric(agg_ga),
        agg_outcome = case_when(
            agg_gf > agg_ga ~ "W",
            agg_gf < agg_ga ~ "L",
            agg_gf == agg_ga ~ "D",
            .default = agg_outcome
        ),
        agg_score = case_when(
            agg_outcome == "W" & venue == "A" ~ paste0(str_sub(agg_score, 3, 3), "-", str_sub(agg_score, 1, 1)),
            .default = agg_score
        ),
    ) %>%
    select(
        game_url,
        ko_time,
        cup_round, cup_leg, cup_stage, cup_replay, cup_section, 
        extra_time,
        pen_outcome, pen_score, pen_gf, pen_ga,
        agg_outcome, agg_score, agg_gf, agg_ga,
        away_goal_outcome
    )

In [50]:
df_left <- results %>%
    rename(game_url = source_url)

df_right <- df2

In [51]:
df_left %>%
    inner_join(df_right, by = "game_url")

season,game_date,opposition,venue,score,home_team,away_team,outcome,home_goals,away_goals,⋯,extra_time,pen_outcome,pen_score,pen_gf,pen_ga,agg_outcome,agg_score,agg_gf,agg_ga,away_goal_outcome
<chr>,<date>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,⋯,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>
2022/23,2023-03-25,Colchester United,A,1-1,Colchester United,Tranmere Rovers,D,1,1,⋯,0,NA,NA,NA,NA,NA,NA,NA,NA,NA
2022/23,2023-03-18,Newport County,H,1-3,Tranmere Rovers,Newport County,L,1,3,⋯,0,NA,NA,NA,NA,NA,NA,NA,NA,NA
2022/23,2023-03-11,Gillingham,A,0-2,Gillingham,Tranmere Rovers,L,2,0,⋯,0,NA,NA,NA,NA,NA,NA,NA,NA,NA
2022/23,2023-03-04,Hartlepool United,H,1-1,Tranmere Rovers,Hartlepool United,D,1,1,⋯,0,NA,NA,NA,NA,NA,NA,NA,NA,NA
2022/23,2023-02-28,Crawley Town,H,1-0,Tranmere Rovers,Crawley Town,W,1,0,⋯,0,NA,NA,NA,NA,NA,NA,NA,NA,NA
2022/23,2023-02-25,Stevenage,A,1-0,Stevenage,Tranmere Rovers,W,0,1,⋯,0,NA,NA,NA,NA,NA,NA,NA,NA,NA
2022/23,2023-02-18,Mansfield Town,H,0-2,Tranmere Rovers,Mansfield Town,L,0,2,⋯,0,NA,NA,NA,NA,NA,NA,NA,NA,NA
2022/23,2023-02-14,Bradford City,A,0-2,Bradford City,Tranmere Rovers,L,2,0,⋯,0,NA,NA,NA,NA,NA,NA,NA,NA,NA
2022/23,2023-02-11,Salford City,H,1-0,Tranmere Rovers,Salford City,W,1,0,⋯,0,NA,NA,NA,NA,NA,NA,NA,NA,NA
